# Import data and general packages

In [11]:
import numpy as np
import os
import pandas as pd


In [10]:
os.chdir("/home/andi/code/mochiyam/simply-japanese/data/2_RawData") # Change user name to run on your machine
os.getcwd()

'/home/andi/code/mochiyam/simply-japanese/data/2_RawData'

In [12]:
data = "Combined_85K_150.xlsx"
df = pd.read_excel(data)

In [20]:
df.head()

,#日本語(原文),#やさしい日本語,#英語(原文)
0,彼女は通りを横切った。,彼女は通りを横に通っていった。,she came across the street .
1,私が知るかぎり彼女は大変よい人だ。,私が知る限り彼女は大変よい人だ。,"as far as i know , she is a very good person ."
2,私のクラスの少女たちはみんな親切だ。,私のクラスの少女たちはみんな親切だ。,all the girls in my class are kind .
3,彼は試験に合格できなかった。,彼は試験に合格できなかった。,he couldn 't pass the examination .
4,彼女はあなたにあえて喜ぶでしょうね。,彼女はあなたに会うことができて喜ぶでしょうね。,"she 'll be glad to see you , won 't she ?"


In [24]:
X = df[df.keys()[0]]
y = df[df.keys()[1]]

In [25]:
X[0], y[0]

('彼女は通りを横切った。', '彼女は通りを横に通っていった。')

# Janome

In [1]:
from janome.tokenizer import Tokenizer

Simple Tokenizer

In [27]:
t = Tokenizer()
for token in t.tokenize(X[0]):
    print(token)

彼女	名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
通り	名詞,一般,*,*,*,*,通り,トオリ,トーリ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
横切っ	動詞,自立,*,*,五段・ラ行,連用タ接続,横切る,ヨコギッ,ヨコギッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。


Even simpler tokenizer<br>
wakati (分ち書き）returns only surface tokens

In [38]:
for token in t.tokenize(X[0], wakati=True):
    print(token)

彼女
は
通り
を
横切っ
た
。


# Identify words to be replaced / ignored